##Task 1. Create a cluster and deploy your app

In [ ]:
export ZONE=us-west1-a
export CLUSTER=onlineboutique-cluster-424
export POOL=optimized-pool-6690

In [ ]:
gcloud container clusters create $CLUSTER \
  --project=${DEVSHELL_PROJECT_ID} --zone $ZONE \
   --machine-type=e2-standard-2 --num-nodes=2

In [ ]:
kubectl create namespace dev
kubectl create namespace prod

In [ ]:
kubectl config set-context --current --namespace dev

In [ ]:
git clone https://github.com/GoogleCloudPlatform/microservices-demo.git
cd microservices-demo
kubectl apply -f ./release/kubernetes-manifests.yaml --namespace dev

In [ ]:
kubectl get svc -w --namespace dev

##Task 2. Migrate to an optimized node pool

In [ ]:
gcloud container node-pools create $POOL \
  --cluster $CLUSTER \
  --machine-type=custom-2-3584 \
  --num-nodes=2 \
  --zone $ZONE

In [ ]:
for node in $(kubectl get nodes -l cloud.google.com/gke-nodepool=default-pool -o=name); do
   kubectl cordon "$node";
done

for node in $(kubectl get nodes -l cloud.google.com/gke-nodepool=default-pool -o=name); do
   kubectl drain --force --ignore-daemonsets --delete-local-data --grace-period=10 "$node";
done

In [ ]:
gcloud container node-pools delete default-pool \
   --cluster $CLUSTER --zone $ZONE

##Task 3. Apply a frontend update

In [ ]:
kubectl create poddisruptionbudget onlineboutique-frontend-pdb \
--selector app=frontend --min-available 1  --namespace dev

In [ ]:
KUBE_EDITOR="nano" kubectl edit deployment/frontend --namespace dev
# image: gcr.io/qwiklabs-resources/onlineboutique-frontend:v2.1
# imagePullPolicy: Always

##Task 4. Autoscale from estimated traffic

In [ ]:
kubectl autoscale deployment frontend --cpu-percent=50 \
   --min=1 --max=11 --namespace dev

In [ ]:
kubectl get hpa --namespace dev

gcloud beta container clusters update $CLUSTER \
   --enable-autoscaling --min-nodes 1 --max-nodes 6 --zone $ZONE

##Task 5. (Optional) Optimize other services